In [1]:
from tree import *
from nv_tree_utilities import *
import pandas as pd

import mkl
mkl.set_num_threads(1)



1

# run opti

In [21]:
def run(par):    
    
    ## load yaz steak data
    data = pd.read_csv('yaz_steak.csv').values
    data = data[:,1:]

    X = data[:,0:24]
    Y = data[:,24] ## Demand for steak is found in column "STEAK"
    cu,co = 9,1
    X_train, X_test, Y_train, Y_test = X[range(0,600),:], X[range(600,760),:], Y[range(0,600)], Y[range(600,760)]

    X_train = X_train.reshape(X_train.shape[0],X_train.shape[1])
    X_test = X_test.reshape(X_test.shape[0],X_test.shape[1])
    Y_train = Y_train.reshape(-1,1)
    Y_test = Y_test.reshape(-1,1)


    X_list = [X_train]
    Y_list = [Y_train]


    p_list = [24]
    runs = 1 # 50
    n_jobs = 1 # 50
    n_trees = 10 # 500;
    N_list = [600]
    # Nx_test = 200
    # Ny_test = 2000
    # Ny_train = 1000


    b_list = np.array([par])#, 1.]) ## backlog -- underage
    h_list = np.array([1])#, 0.05]) ## holding cost -- overage
    C = 9999 ## capacity constraint
    L = len(h_list)

    honesty = False; 
    verbose = False; oracle = True;
    bootstrap = True; 

    cond_mean = [lambda x: 3]#, lambda x: 3]
    cond_std = [lambda x: np.exp(x[:, 0])]#, lambda x: np.exp(x[:, 1])]

    risk_all = {}
    feature_split_all = {}
    results_eval_all = {}

    direct = ''
    date = ''
    output = "nv_n.txt"

    with open(output, 'w') as f:
        print("Parameters set", file = f)

    with open(output, 'w') as f:
        print("start", file = f)

    for N in N_list:
        risk_all[str(N)] = {}
        feature_split_all[str(N)] = {}
        results_eval_all[str(N)] = {}

        for p in p_list:
            with open(output, 'a') as f:
                print("N: ", N, file = f)
                print("p: ", p, file = f)

            n_proposals = N; 
            mtry = p;
            subsample_ratio = 1;
            max_depth=100; 
            min_leaf_size=10; 
            balancedness_tol = 0.2; 

            time1 = time.time()
            results_fit = Parallel(n_jobs=n_jobs, verbose = 3)(delayed(compare_forest_one_run)(X_list[run], Y_list[run], X_list[run], Y_list[run], 
                h_list = h_list, b_list = b_list, C = C, 
                n_trees = n_trees, honesty= honesty, mtry = mtry, subsample_ratio = subsample_ratio, oracle = oracle, min_leaf_size = min_leaf_size, verbose = verbose, max_depth = max_depth, n_proposals = n_proposals, balancedness_tol = balancedness_tol, bootstrap = bootstrap) for run in range(runs))
            time2 = time.time()
            with open(output, 'a') as f:
                print("time: ", time2 - time1, file = f)
                print("------------------------", file = f)

            (decisions,risk) = evaluate_one_run_original(results_fit[0],X_test, Y_test, Y_train, h_list =h_list, b_list = b_list, C = C, verbose = False)
    return (par,decisions,risk)

In [22]:
params = [19, 9, 5.666666667,4,3,2.333333333,1.857142857,1.5,1.222222222,1,0.818181818,
          0.666666667,0.538461538,0.428571429,0.333333333,0.25,0.176470588,0.111111111,0.052631579]


In [ ]:
# res = run(9)


from multiprocessing import Pool

pool = Pool(processes=12)
results = pool.map(run,params)
pool.close()
pool.join()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [ ]:
df = pd.DataFrame()
for run in range(len(results)):
    # 0: overage, 1:decision, 2:risk
    over = results[run][0]
    risk = results[run][2]
    tmp = pd.DataFrame({'over':over},index=[0])
    for k,r in risk.items():
        tmp[k] = r.mean()
    df = df.append(tmp)
df.reset_index(drop=True)
df.to_csv('SOF_results.csv')
    

In [11]:
for k,r in risk.items():
    print('{}: {}'.format(k,r.mean()))


({'rf_approx_sol': array([[24.       ],
         [28.       ],
         [31.       ],
         [25.       ],
         [32.       ],
         [54.0308642],
         [24.       ],
         [24.       ],
         [28.       ],
         [31.       ],
         [25.       ],
         [32.       ],
         [54.0308642],
         [24.       ],
         [24.       ],
         [28.       ],
         [31.       ],
         [25.       ],
         [32.       ],
         [54.0308642],
         [24.       ],
         [24.       ],
         [28.       ],
         [31.       ],
         [25.       ],
         [32.       ],
         [54.0308642],
         [24.       ],
         [24.       ],
         [28.       ],
         [31.       ],
         [25.       ],
         [32.       ],
         [54.0308642],
         [24.       ],
         [24.       ],
         [28.       ],
         [31.       ],
         [25.       ],
         [32.       ],
         [54.0308642],
         [24.       ],
         [24.    